In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%pip -q install git+https://github.com/thunni-noi/pytube.git
%pip -q install pip install youtube_transcript_api
%pip -q install moviepy
%pip -q install pydub
#modified version of pytube to support current youtube thingy
exit()

In [6]:
import urllib.parse
import re
from tqdm.notebook import tqdm_notebook

def url_to_watch_id(url):
  vid_list = []
  pattern_watch = 'https://www.youtube.com/watch?'
  pattern_short = 'https://youtu.be/'

    #When using a normal URL
  if re.match(pattern_watch,url):
    yturl_qs = urllib.parse.urlparse(url).query
    vid = urllib.parse.parse_qs(yturl_qs)['v'][0]
    watch_id = vid

    #For shortened URLs
  elif re.match(pattern_short,url):
    # "https://youtu.be/"The 11 characters following the video ID
    vid = url[17:28]
    watch_id = vid

  else:
    print('error:\n URL is\"https://www.youtube.com/watch?\"Or')
    print('  \"https://youtu.be/\"Please specify the URL that starts with.')
    print('  - '+ str(i+1)+ 'Item:' + url)
  return watch_id

In [7]:
from pytube import YouTube
import shutil

def name_gen(path): #UNUSED
  #if not name.endswith('.mp4') : filename = name+ '.mp4'
  serial_num = 0
  name = str(serial_num)
  while os.path.exists(path+name+'.mp4') :
    print(path+name+'.mp4 already exists!')
    serial_num += 1
    name = str(serial_num)
    print(f'Trying {name}.mp4....')
  return name + '.mp4', serial_num

def download_video(url, path, vid_dw_number):
  yt = YouTube(url)
  filename = f'{vid_dw_number}.mp4'
  if os.path.exists(os.path.join(path, filename)) : os.remove(os.path.join(path, filename))
  serial_num = vid_dw_number
  my_streams = yt.streams.filter(file_extension='mp4', progressive = True)
  itag_list = []
  res_list = []
  for streams in my_streams :
    itag_list.append(streams.itag)
    res_list.append(int((streams.resolution).replace('p', '')))
    #print(f"Video itag : {streams.itag} Resolution : {streams.resolution} VCodec : {streams.codecs[0]}")
  best_res = min(res_list)
  best_res_index = res_list.index(best_res)
  print(f'Resolution : {best_res}, Itag : {itag_list[best_res_index]}')
  video = yt.streams.get_by_itag(itag_list[best_res_index])
  video.download(output_path = './' , filename = filename)
  print(f'Downloaded {yt.title} as {filename}')
  shutil.move(f'./{filename}', str(path))
  output_path = os.path.join(path, filename)
  return output_path, serial_num, url

In [8]:
from youtube_transcript_api import YouTubeTranscriptApi
from datetime import datetime, timedelta
from moviepy.editor import *
from pydub import AudioSegment
import random
import os


def video_to_mp3(input_path, output_path, serial):
    output_path = os.path.join(output_path, str(serial))
    if not os.path.exists(output_path):
        print(f'{output_path} is not exists! Creating!!!')
        os.makedirs(output_path)
    video = VideoFileClip(input_path)
    video.audio.write_audiofile(os.path.join(output_path, 'full.wav'))
    return output_path
  

  
def audio_breaker_timestamp(basepath, outputpath, timestamp_list):
    audio_speech_list = []
    baseAudio = AudioSegment.from_wav(basepath)
    speech_order = 0
    for startNend in timestamp_list:
        cStart, cEnd = startNend.split('~')
        newAudio = baseAudio[int(cStart):int(cEnd)]
        newAudio.export(os.path.join(outputpath, f'{str(speech_order)}.mp3'), format = 'mp3')
        audio_speech_list.append(os.path.join(outputpath, f'{str(speech_order)}.mp3'))
        #print(f'Creating {speech_order}.wav!')
        speech_order += 1
    return audio_speech_list

In [9]:
from datetime import datetime, timedelta
def time_format(start, duration):
  remove = bool(False)
  start = str(start)
  dur = str(duration)
  while len(start.split('.')[1]) < 3 : start += '0'
  while len(dur.split('.')[1]) < 3 : dur += '0'
  start = int(start.replace('.',''))
  dur = int(dur.replace('.',''))
  if dur >= 1001 : remove = False
  else : remove = True
  startmilli = start
  endmilli = dur + start
  end = str(timedelta(milliseconds= (start + dur)))
  start = str(timedelta(milliseconds=start))
  start = (str(start)[2:])[:-3]
  end = (str(end)[2:])[:-3]
  return start, end, startmilli, endmilli, remove

def transcript_file_gen(serial_num, transcript, path):
  serial_num = str(serial_num)
  txt_file = open(os.path.join(path, serial_num + '.txt'), 'w+')
  output_path = (os.path.join(path, serial_num + 'txt'))
  timestamp_list =  []
  transcript_list = []
  for seq in transcript:
    txt = seq['text']
    txt = txt.replace('\u200b', '').replace('\n', '').strip()
    start = seq['start']
    dur = seq['duration']
    
    if '：' in txt : txt = txt.split('：')[-1] 
    sStart, sEnd, tStart, tEnd, remove = time_format(start, dur)
    if remove == True :
      #print('skipped!')
      continue
    txt_file.write(f'{sStart} || {sEnd} || {txt} \n')
    timestamp_list.append(f'{str(tStart)}~{str(tEnd)}')
    transcript_list.append(txt)
    #print(txt)
  return output_path,timestamp_list, transcript_list
  #print(time_format(start).second)
  #print(f'Text : {txt}')

In [11]:
#config!
pl = Playlist('YOUTUBE_PLAYLIST_LINK') #change playlists urls here
datasets_name = str('NAME') #change datasets name here
# CAUTION : DO NOT SET THE NAME TO HAVE THE WORD 'CLEAN' OR SET DUPLICATED NAME

In [12]:
import os
from pytube import Playlist
pickle_folder_path = '/content/drive/MyDrive/livetranslator/pickle'
datasets_path = f'/content/drive/MyDrive/livetranslator/{datasets_name}_datasets/'
video_folder_path = os.path.join(datasets_path, 'video')
audio_folder_path = os.path.join(datasets_path, 'audio')
transcript_folder_path = os.path.join(datasets_path, 'transcript')

In [13]:
#remove file
import shutil
if not os.path.exists(datasets_path) : os.makedirs(datasets_path)
for f in os.listdir(datasets_path):
    temppath = os.path.join(datasets_path, f)
    try : 
        shutil.rmtree(temppath)
    except OSError:
        os.remove(temppath)
temppath = 'NA'
del temppath
os.makedirs(video_folder_path)
os.makedirs(audio_folder_path)
os.makedirs(transcript_folder_path)

In [ ]:
import pandas as pd
import time
import datetime

urls = list(pl.video_urls)
dataframe = pd.DataFrame(columns = ['path', 'sentence'])
error_unuse = 0
if not datasets_name.endswith('_') : str(datasets_name) + str('_')
vid_info = []
for temp_index in tqdm_notebook(range(0,len(urls)), total = len(urls), desc = 'Downloading and extracing audio/transcript from playlists : '):
  try :
    transcript= YouTubeTranscriptApi.get_transcript(url_to_watch_id(urls[temp_index]), languages=['ja'])
  except Exception as err :
    print(f'Error, Something went wrong, Error')
    line_noti_msg(f'Found error : skipping {err}')
    error_unuse += 1
    continue
  index = temp_index - error_unuse
  download_watch_id = url_to_watch_id(urls[index])
  try :
    download_path, download_number, download_url = download_video(str(urls[index]), video_folder_path, index)
  except Exception as err :
    print(f'Error, Something went wrong, Error')
    line_noti_msg(f'Found error : skipping {err}')
    error_unuse += 1
    continue
  
  temp_dict = {'url' : download_url,'watch_id' : download_watch_id,'video_path' : download_path, 'serial_number' : download_number }
  vid_info.append(temp_dict)
  del temp_dict
  
  transcript_textpath, transcript_timestamp_list, transcript_sub_list = transcript_file_gen(index, transcript, transcript_folder_path)
  vid_info[index]['transcript_path'] = transcript_textpath
  vid_info[index]['timestamp'] = transcript_timestamp_list
  vid_info[index]['transcript_list'] = transcript_sub_list
  vid_info[index]['audio_path'] = video_to_mp3(vid_info[index]['video_path'],audio_folder_path , vid_info[index]['serial_number'])
  vid_info[index]['speech_list'] = audio_breaker_timestamp(
    os.path.join(vid_info[index]['audio_path'], 'full.wav'), 
    os.path.join(vid_info[index]['audio_path']), 
    vid_info[index]['timestamp'])
  for i, speech in enumerate(vid_info[index]['speech_list']):
    dataframe = dataframe.append({'path' : speech, 'sentence' : vid_info[index]['transcript_list'][i]},ignore_index = True)

dataframe = dataframe.drop(dataframe[dataframe['sentence'] == ''].index) #drop if subtitle is empty
dataframe.to_pickle(os.path.join(pickle_folder_path, f'{datasets_name}.pkl'))
  #df_train.to_excel(os.path.join(datasets_path, f'{datasets_name}_test.xlsx'))
print(f'Finished downloading datasets. Total : {len(dataframe)} audio were extracted!')
